In [102]:
import pandas as pd
from sqlalchemy import create_engine


### Extract CSVs into DataFrames

In [103]:
# import the imdb data
imdb_file = "IMDb movies.csv"
imdb_df = pd.read_csv(imdb_file)
imdb_df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0


In [104]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81273 entries, 0 to 81272
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          81273 non-null  object 
 1   title                  81273 non-null  object 
 2   original_title         81273 non-null  object 
 3   year                   81273 non-null  int64  
 4   date_published         81273 non-null  object 
 5   genre                  81273 non-null  object 
 6   duration               81273 non-null  int64  
 7   country                81234 non-null  object 
 8   language               80518 non-null  object 
 9   director               81200 non-null  object 
 10  writer                 79780 non-null  object 
 11  production_company     76948 non-null  object 
 12  actors                 81207 non-null  object 
 13  description            78843 non-null  object 
 14  avg_vote               81273 non-null  float64
 15  vo

In [105]:
# import the netflix data
netflix_file = "netflix_titles.csv"
netflix_df = pd.read_csv(netflix_file)
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


In [106]:
#split the netflix data intp movies and TV shows
netflix_movies_df = netflix_df.loc[netflix_df['type']=='Movie',:]
netflix_movies_df.info()
netflix_TV_df = netflix_df.loc[netflix_df['type']=='TV Show']
netflix_TV_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4265 entries, 0 to 6231
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       4265 non-null   int64 
 1   type          4265 non-null   object
 2   title         4265 non-null   object
 3   director      4137 non-null   object
 4   cast          3905 non-null   object
 5   country       4070 non-null   object
 6   date_added    4264 non-null   object
 7   release_year  4265 non-null   int64 
 8   rating        4257 non-null   object
 9   duration      4265 non-null   object
 10  listed_in     4265 non-null   object
 11  description   4265 non-null   object
dtypes: int64(2), object(10)
memory usage: 433.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1969 entries, 2 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1969 non-null   int64 
 1   type          

In [107]:
# after checking the data we decided to merge on directors + title
# to avoid mismatches in the data
#check for missing directors in the data
bool_series = pd.isnull(netflix_movies_df["director"])  
print (f"There are {len(netflix_movies_df[bool_series])} movies with no director information") 
bool_series = pd.isnull(netflix_TV_df["director"])  
print (f"There are {len(netflix_TV_df[bool_series])} TV Shows with no director information") 
 

There are 128 movies with no director information
There are 1841 TV Shows with no director information


In [108]:
#There are very few tv shows with matching data
combined_TV_df = pd.merge(netflix_TV_df, imdb_df, left_on=['title'], 
                    right_on=['title'], suffixes=('_net', '_imdb') )

print (f" There are {len(combined_TV_df)} matching TV shows that have director data")

 There are 409 matching TV shows that have director data


In [109]:
# merge the two data sets on title and director
combined_movie_df = pd.merge(netflix_movies_df, imdb_df, left_on=['title','director'], 
                    right_on=['title','director'], suffixes=('_net', '_imdb') )
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1800 entries, 0 to 1799
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   show_id                1800 non-null   int64  
 1   type                   1800 non-null   object 
 2   title                  1800 non-null   object 
 3   director               1800 non-null   object 
 4   cast                   1798 non-null   object 
 5   country_net            1796 non-null   object 
 6   date_added             1800 non-null   object 
 7   release_year           1800 non-null   int64  
 8   rating                 1799 non-null   object 
 9   duration_net           1800 non-null   object 
 10  listed_in              1800 non-null   object 
 11  description_net        1800 non-null   object 
 12  imdb_title_id          1800 non-null   object 
 13  original_title         1800 non-null   object 
 14  year                   1800 non-null   int64  
 15  date

In [110]:
#check for mismatches between the two data sets
#different movies with the same title

combined_movie_df["title"].value_counts()


The Silence                        3
Benji                              2
The Lovers                         2
The Outsider                       2
Frank and Cindy                    2
                                  ..
Chameli                            1
The Cloverfield Paradox            1
Scream 2                           1
Barsaat                            1
The Goods: Live Hard, Sell Hard    1
Name: title, Length: 1785, dtype: int64

In [111]:
# there are 15 entries that are duplicated
#probably because imdb has no director information
pd.options.display.max_rows = 1000 #this allows for full printing
duplicateDFRow = combined_movie_df[combined_movie_df["title"].duplicated()]
print(len(duplicateDFRow))
print(duplicateDFRow)

15
       show_id   type                title          director  \
198   81021082  Movie           Kuntilanak   Rizal Mantovani   
388   81189912  Movie                Drive  Tarun Mansukhani   
602   80152434  Movie         The Outsider  Martin Zandvliet   
668   81072516  Movie               Sarkar   A.R. Murugadoss   
682   80204923  Movie                Benji      Brandon Camp   
777   80238292  Movie          The Silence    Gajendra Ahire   
1112  80245041  Movie                 Solo       Hugo Stuven   
1259  80038446  Movie           The Lovers      Roland Joffé   
1279  80214583  Movie  People You May Know   Sherwin Shilati   
1301  80209153  Movie          Blood Money       Lucky McKee   
1483  70184051  Movie              One Day     Lone Scherfig   
1490  70059026  Movie                  Don     Farhan Akhtar   
1510  80093812  Movie                 Zoom     Pedro Morelli   
1757  81021447  Movie          The Silence  John R. Leonetti   
1799  80085439  Movie      Frank and 

In [112]:
#We drop the duplicated items

combined_movie_df = combined_movie_df.drop_duplicates(subset = ["title"])

combined_movie_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1785 entries, 0 to 1798
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   show_id                1785 non-null   int64  
 1   type                   1785 non-null   object 
 2   title                  1785 non-null   object 
 3   director               1785 non-null   object 
 4   cast                   1783 non-null   object 
 5   country_net            1781 non-null   object 
 6   date_added             1785 non-null   object 
 7   release_year           1785 non-null   int64  
 8   rating                 1784 non-null   object 
 9   duration_net           1785 non-null   object 
 10  listed_in              1785 non-null   object 
 11  description_net        1785 non-null   object 
 12  imdb_title_id          1785 non-null   object 
 13  original_title         1785 non-null   object 
 14  year                   1785 non-null   int64  
 15  date

In [113]:
# cut the columns that are missing a lot of information
#27 budget # 28 usa_gross_income #29 worlwide_gross_income #30 object metascore             
combined_movie_df = combined_movie_df[['show_id', 'type', 'title',
   'cast', 'country_net', 'date_added', 'release_year', 'rating', 'duration_net',
'listed_in', 'description_net', 'imdb_title_id', 'original_title', 'year',
'date_published', 'genre', 'duration_imdb', 'country_imdb', 'language', 'actors', 'description_imdb', 'avg_vote',
'votes']].copy()
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1785 entries, 0 to 1798
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   show_id           1785 non-null   int64  
 1   type              1785 non-null   object 
 2   title             1785 non-null   object 
 3   cast              1783 non-null   object 
 4   country_net       1781 non-null   object 
 5   date_added        1785 non-null   object 
 6   release_year      1785 non-null   int64  
 7   rating            1784 non-null   object 
 8   duration_net      1785 non-null   object 
 9   listed_in         1785 non-null   object 
 10  description_net   1785 non-null   object 
 11  imdb_title_id     1785 non-null   object 
 12  original_title    1785 non-null   object 
 13  year              1785 non-null   int64  
 14  date_published    1785 non-null   object 
 15  genre             1785 non-null   object 
 16  duration_imdb     1785 non-null   int64  


In [114]:
#actors and cast are the same information, but cast is missing some data
# the two durations have the same infomation, but imdb duration is integer
#and netflix is missing some data, so we cut netflix
# release year and date published are the same, so we cut date_published

combined_movie_df = combined_movie_df[['show_id', 'type', 'title', 'date_added',
  'country_net' ,'release_year', 'rating',
'listed_in', 'description_net', 'imdb_title_id', 'original_title', 'year',
      'genre', 'duration_imdb',
'country_imdb', 'language', 'actors', 'description_imdb', 'avg_vote',
'votes']].copy()
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1785 entries, 0 to 1798
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   show_id           1785 non-null   int64  
 1   type              1785 non-null   object 
 2   title             1785 non-null   object 
 3   date_added        1785 non-null   object 
 4   country_net       1781 non-null   object 
 5   release_year      1785 non-null   int64  
 6   rating            1784 non-null   object 
 7   listed_in         1785 non-null   object 
 8   description_net   1785 non-null   object 
 9   imdb_title_id     1785 non-null   object 
 10  original_title    1785 non-null   object 
 11  year              1785 non-null   int64  
 12  genre             1785 non-null   object 
 13  duration_imdb     1785 non-null   int64  
 14  country_imdb      1784 non-null   object 
 15  language          1771 non-null   object 
 16  actors            1785 non-null   object 


In [115]:
#Some columns are duplicated, so we check if we can complete the information
#by combining the columns
#Check if the country null values are missing in both columns
bool_series = pd.isnull(combined_movie_df['country_net']) 
bool_series = pd.isnull(combined_movie_df['country_imdb'])  
combined_movie_df[bool_series].T  

,138
show_id,80211622
type,Movie
title,Been So Long
date_added,"October 26, 2018"
country_net,NaN
release_year,2018
rating,TV-MA
listed_in,"Dramas, International Movies, Music & Musicals"
description_net,A single mother in London's Camden Town hears ...
imdb_title_id,tt1682886


In [94]:
# the imdb database has more country data so we cut country_net
#both datasets are missing one piece of data so we replace it with not available
 
combined_movie_df["country_imdb"].fillna("Not Available", inplace = True)

combined_movie_df = combined_movie_df[['show_id', 'type', 'title', 'date_added', 'release_year', 'rating',
'listed_in', 'description_net', 'imdb_title_id', 'original_title', 'year', 'genre', 'duration_imdb', 'country_imdb', 'language', 'actors', 'description_imdb', 'avg_vote',
'votes']].copy()
combined_movie_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1785 entries, 0 to 1798
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   show_id           1785 non-null   int64  
 1   type              1785 non-null   object 
 2   title             1785 non-null   object 
 3   date_added        1785 non-null   object 
 4   release_year      1785 non-null   int64  
 5   rating            1785 non-null   object 
 6   listed_in         1785 non-null   object 
 7   description_net   1785 non-null   object 
 8   imdb_title_id     1785 non-null   object 
 9   original_title    1785 non-null   object 
 10  year              1785 non-null   int64  
 11  genre             1785 non-null   object 
 12  duration_imdb     1785 non-null   int64  
 13  country_imdb      1785 non-null   object 
 14  language          1771 non-null   object 
 15  actors            1785 non-null   object 
 16  description_imdb  1785 non-null   object 


In [116]:
# fill the missing rating as "not available"
combined_movie_df["rating"].fillna("Not Available", inplace = True)

In [124]:
combined_movie_df[['year','release_year']]



,year,release_year
0,2019,2019
1,2014,2014
2,2015,2015
3,2015,2015
4,2015,2014
...,...,...
1794,2016,2016
1795,2015,2015
1796,2014,2014
1797,2016,2016


In [134]:
# gains = df[df.buy_price < df.sell_price]
different = combined_movie_df.year != combined_movie_df.release_year
different.value_counts()


False    1701
True       84
dtype: int64

In [137]:
combined_movie_df.head(1798)

,show_id,type,title,date_added,country_net,release_year,rating,listed_in,description_net,imdb_title_id,original_title,year,genre,duration_imdb,country_imdb,language,actors,description_imdb,avg_vote,votes
0,81145628,Movie,Norm of the North: King Sized Adventure,"September 9, 2019","United States, India, South Korea, China",2019,TV-PG,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,tt9428190,Norm of the North: King Sized Adventure,2019,"Animation, Adventure, Comedy",90,"USA, India, South Korea, China",English,"Jennifer Cameron, Brian Dobson, Michael Dobson...",An ancient Chinese artifact has been stolen by...,3.2,185
1,70304990,Movie,Good People,"September 8, 2017","United States, United Kingdom, Denmark, Sweden",2014,R,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,tt1361318,Good People,2014,"Action, Crime, Thriller",90,"USA, UK, Denmark, Sweden",English,"Sam Spruell, Michael Fox, Diarmaid Murtagh, Fr...",Discovering a stash of cash in their dead tena...,5.5,14655
2,70299204,Movie,Kidnapping Mr. Heineken,"September 8, 2017","Netherlands, Belgium, United Kingdom, United S...",2015,R,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ...",tt2917388,Kidnapping Mr. Heineken,2015,"Action, Crime, Drama",95,"Netherlands, Belgium, UK, USA","English, Dutch, German","Jim Sturgess, Sam Worthington, Ryan Kwanten, A...","The inside story of the planning, execution, r...",6.1,18168
3,80057969,Movie,Love,"September 8, 2017","France, Belgium",2015,NR,"Cult Movies, Dramas, Independent Movies",A man in an unsatisfying marriage recalls the ...,tt3774694,Love,2015,"Drama, Romance",135,"France, Belgium","English, French","Aomi Muyock, Karl Glusman, Klara Kristin, Ugo ...",Murphy is an American living in Paris who ente...,6.1,39385
4,80060297,Movie,Manhattan Romance,"September 8, 2017",United States,2014,TV-14,"Comedies, Independent Movies, Romantic Movies",A filmmaker working on a documentary about lov...,tt2608324,Manhattan Romance,2015,"Comedy, Drama, Romance",94,USA,English,"Arnold C. Baker II, Jessie Barr, Tommy Burke, ...","Danny, a commercial editor and documentary fil...",5.5,1082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,80115101,Movie,Deuces,"April 1, 2017",United States,2016,TV-MA,Dramas,Assigned to go undercover to take down an impe...,tt5081606,Deuces,2016,"Action, Crime, Drama",87,USA,English,"Larenz Tate, Meagan Good, Rotimi, Rick Gonzale...",An undercover agent infiltrates a crime ring r...,5.9,790
1795,80081955,Movie,Five Nights in Maine,"April 1, 2017",United States,2015,TV-14,"Dramas, Independent Movies","After his wife dies in a car accident, a grief...",tt3352034,Five Nights in Maine,2015,Drama,82,USA,English,"David Oyelowo, Hani Furstenberg, Neal Lerner, ...","A young African American man, reeling from the...",4.5,617
1796,70302835,Movie,Killa,"April 1, 2017",India,2014,TV-14,"Dramas, International Movies",Growing up poses challenges for Chinu when he ...,tt3341582,Killa,2014,"Comedy, Drama",107,India,Marathi,"Amruta Subhash, Archit Deodhar, Parth Bhalerao...",A 11-year-old Indian boy who has just lost his...,8.0,1170
1797,80125512,Movie,Lavender,"April 1, 2017","Canada, United States",2016,NR,"Dramas, International Movies, Thrillers","After discovering old fractures in her skull, ...",tt4680980,Lavender,2016,"Drama, Thriller",92,"Canada, USA",English,"Abbie Cornish, Dermot Mulroney, Justin Long, L...","After discovering old fractures in her skull, ...",5.3,5083


In [ ]:
# if we need to find additional data we can use this package
# pip install imdbpy

In [ ]:
# This is the code we used in class for cleaning data and exporting it

### Transform premise DataFrame

In [ ]:
new_premise_df = premise_df[['Premises Name', 'County ID Code']].copy()
new_premise_df.head()

In [ ]:
new_premise_df.rename(columns = {'Premises Name':'premise_name'}, inplace = True) 

In [ ]:
new_premise_df.rename(columns = {'County ID Code':'county_id'}, inplace = True) 
new_premise_df.duplicated()
new_premise_df.drop_duplicates(['premise_name'])

### Transform county DataFrame

In [ ]:
county_df.head()
new_county_df = county_df[['County Name (Licensee)', 'County ID Code', 'License Count']].copy()
new_county_df.count()



In [ ]:
new_county_df.rename(columns = {'License Count':'license_count',
                               "County Name (Licensee)": "county_name",
                               "County ID Code": "county_id"}, inplace = True) 

# A `county` table that contains the columns `id`, `county_name`, 
#     `license_count` and `county_id`.
new_county_df

### Create database connection

In [ ]:

rds_connection_string = "postgres:1117@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Confirm tables
engine.table_names()


### Load DataFrames into database

In [ ]:
new_county_df.to_sql(name='county', con=engine, if_exists='append', index=False)

In [ ]:
new_premise_df.to_sql(name='premise', con=engine, if_exists='append', index=False)